In [ ]:
# dataset.py
import torch
import numpy as np

from PIL import Image
from PIL import ImageFile


ImageFile.LOAD_TRUNCATED_IMAGES = True

class ClassificationDataset:
    def __init__(self, image_paths, targets, resize=None, augmentations=None):
        self.image_paths = image_paths
        self.targets = targets
        self.resize = resize
        self.augmentations = augmentations
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, item):
        image = Image.open(self.image_paths[item])
        image = image.convert("RGB")
        targets = self.targets[item]
        
        if self.resize is not None:
            image = image.resize(
                (self.resize[1], self.resize[0]),
            resample=Image.BILINEAR
            )
        image = np.array(image)
        
        if self.augmentations is not None:
            augmented = self.augmentations(image = image)
            image = augmented["image"]
        
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        return {
            'image': torch.tensor(image, dtype=torch.float),
            'targets': torch.tensor(targets, dtype=torch.long),
        }

In [ ]:
# engine.py

import torch
import torch.nn as nn

from tqdm import tqdm

def train(dataset, data_loader, model, optimizer, device):
    model.train()
    num_batches = int(len(dataset)/data_loader.batch_size)
    tk0 = tqdm(data_loader, total=num_batches)
    for d in tk0:
        inputs = data['image']
        targets = data['targets']
        inputs = inputs.to(device, dtype=torch.float)
        targets = targets.to(device, dtype=torch.float)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = nn.BCEWithLogitsLoss()(outputs, targets.view(-1,1))
        loss.backward()
        optimizer.step()
    
def evaluate(data_loader, model, device):
    model.eval()
    final_targets = []
    final_outputs = []
    
    with torch.no_grad():
        for data in data_loader:
            inputs = inputs.to(device, dtype=torch.float)
            targets = targets.to(device, dtype=torch.float)
            output = model(inputs)
            targets = targets.detach().cpu().numpy().tolist()
            output = output.detach().cpu().numpy().tolist()
            final_targets.extend(targets)
            final_outputs.extend(output)
    return final_outputs, final_targets

In [ ]:
# !pip install pretrainedmodels -q

In [ ]:
# model.py
import torch.nn as nn
import pretrainedmodels

def get_model(pretrained):
    if pretrained:
        model = pretrainedmodels.__dict__['alexnet'](
            pretrained='imagenet'
        )
    else:
        model = pretrained.__dict__['alexnet'](
            pretrainedmodels=None
        )
    model.last_linear = nn.Sequential(
        nn.BatchNorm1d(4096), 
        nn.Dropout(p=0.25),
        nn.Linear(in_features=4096,out_features=2048),
        nn.ReLU(),
        nn.BatchNorm1d(2048, eps=1e-5, momentum=0.1),
        nn.Dropout(p=0.5),
        nn.Linear(in_features=2048, out_features=1),
        nn.Sigmoid()
    )
    return model

In [ ]:
# train.py
import os

import pandas as pd
import numpy as np

import albumentations
import torch

from sklearn import metrics
from sklearn.model_selection import train_test_split

data_path = '../input/siim-isic-melanoma-classification/jpeg'
device = 'cuda'
epochs = 5
df = pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')
df.head()
images = df.image_name.values.tolist()
images = [
    os.path.join(data_path, 'train/', i+'.jpg') for i in images
]
targets = df.target.values
model = get_model(pretrained=True)
model.to(device)
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
aug = albumentations.Compose(
    [
        albumentations.Normalize(mean, std, max_pixel_value = 255.0
                                )
    ]
)
train_images, valid_images, train_targets, valid_targets = train_test_split(images, targets, stratify=targets, random_state=42)
train_dataset = ClassificationDataset(
image_paths = train_images, 
targets = train_targets,
resize = (227, 227), 
augmentations = aug
)
valid_dataset = ClassificationDataset(
image_paths = valid_images, 
targets = train_targets,
resize = (227, 227), 
augmentations = aug
)
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=16, shuffle=False, num_workers=4
)

valid_loader = torch.utils.data.DataLoader(
    valid_dataset, batch_size=16, shuffle=False, num_workers=4
)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)

for epoch in range(1):
    train(train_dataset,train_loader, model, optimizer, device=device)
    predictions, valid_targets = evaluate(
        valid_loader, model, device=device)
    roc_auc = metrics.roc_auc_score(valid_targets, predictions)
    pritn(f"Epoch={epoch}, Valid ROC AUC = {roc_auc}"
         )